In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

from multiprocess import Pool
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from conversion import images_to_volume

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


# All structures for annotation volume

In [6]:
stack = 'MD594'
downscale = 32

for structure in all_known_structures_sided:
    print structure
    try:
        volume = DataManager.load_score_volume(stack=stack, volume_type='annotation_as_score', 
                                               structure=structure, downscale=downscale)

        t = time.time()

        gy_gx_gz = np.gradient(volume.astype(np.float32), 3, 3, 3)
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important;
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 2s (float32) vs. 20s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (structure, time.time() - t))

        t = time.time()

        gx_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, volume_type='annotation_as_score', suffix='gx')
        gy_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, volume_type='annotation_as_score', suffix='gy')
        gz_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, volume_type='annotation_as_score', suffix='gz')

        create_parent_dir_if_not_exists(gx_fp)

        bp.pack_ndarray_file(gy_gx_gz[1], gx_fp)
        bp.pack_ndarray_file(gy_gx_gz[0], gy_fp)
        bp.pack_ndarray_file(gy_gx_gz[2], gz_fp)

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (structure, time.time() - t))

    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error computing gradient for %s.\n' % structure)

5N_L


Gradient 5N_L: 1.480075 seconds
save 5N_L: 0.265533 seconds


5N_R


Gradient 5N_R: 1.476082 seconds
save 5N_R: 0.217823 seconds


6N_L


...

Gradient RtTg: 1.478077 seconds
save RtTg: 0.216785 seconds
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD594/MD594_down32_annotationAsScoreVolume/score_volumes/MD594_down32_annotationAsScoreVolume_sp5.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD594/MD594_down32_annotationAsScoreVolume/score_volumes/MD594_down32_annotationAsScoreVolume_sp5.bp'
Error computing gradient for sp5.
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD594/MD594_down32_annotationAsScoreVolume/score_volumes/MD594_down32_annotationAsScoreVolume_outerContour.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/MD594/MD594_down32_annotationAsScoreVolume/score_volumes/MD594_down32_annotationAsScoreVolume_outerContour.bp'
Error computing gradient for outerContour.


sp5
outerContour
SC


Gradient SC: 1.474459 seconds
save SC: 0.223324 seconds


IC


Gradient IC: 1.481006 seconds
save IC: 0.234980 seconds


# For classifier score volume

In [2]:
stack = 'MD657'
classifier_setting = 30
downscale = 32

# All structures

In [5]:
for structure in all_known_structures:
    print structure
    try:
        volume = DataManager.load_original_volume(stack=stack, structure=structure, downscale=downscale,
                                               classifier_setting=classifier_setting, volume_type='score')

        t = time.time()

        gy_gx_gz = np.gradient(volume.astype(np.float32), 3, 3, 3)
        # 3.3 second - re-computing is much faster than loading
        # .astype(np.float32) is important;
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 2s (float32) vs. 20s (float16)

        sys.stderr.write('Gradient %s: %f seconds\n' % (structure, time.time() - t)) # 5s

        t = time.time()

        gx_fp = DataManager.get_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_setting, suffix='gx')
        gy_fp = DataManager.get_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_setting, suffix='gy')
        gz_fp = DataManager.get_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_setting, suffix='gz')

#         create_if_not_exists(os.path.dirname(gx_fp))

#         bp.pack_ndarray_file(gy_gx_gz[1], gx_fp)
#         bp.pack_ndarray_file(gy_gx_gz[0], gy_fp)
#         bp.pack_ndarray_file(gy_gx_gz[2], gz_fp)

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (structure, time.time() - t)) # 4-7s

    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error computing gradient for %s.\n' % structure)

File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt


5N


Gradient 5N: 24.655552 seconds
save 5N: 0.074307 seconds


6N


Gradient 6N: 16.155491 seconds
save 6N: 0.072583 seconds


...

Gradient LRt: 15.544566 seconds
save LRt: 0.072003 seconds


Pn


Gradient Pn: 15.526288 seconds
save Pn: 0.071837 seconds


Tz


KeyboardInterrupt: 

# Single structure

In [13]:
try:
    volume = DataManager.load_score_volume(stack=stack, structure=structure, downscale=downscale,
                                           classifier_setting=classifier_setting)

    t = time.time()

    gy_gx_gz = np.gradient(volume.astype(np.float32), 7, 7, )
    # 3.3 second - re-computing is much faster than loading
    # .astype(np.float32) is important;
    # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
    # 2s (float32) vs. 20s (float16)

    sys.stderr.write('Gradient %s: %f seconds\n' % (structure, time.time() - t))

    t = time.time()

#     gx_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_setting, suffix='gx')
#     gy_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_setting, suffix='gy')
#     gz_fp = DataManager.get_score_volume_gradient_filepath(stack=stack, structure=structure, downscale=downscale, classifier_setting=classifier_setting, suffix='gz')

#     create_if_not_exists(os.path.dirname(gx_fp))

#     bp.pack_ndarray_file(gy_gx_gz[1], gx_fp)
#     bp.pack_ndarray_file(gy_gx_gz[0], gy_fp)
#     bp.pack_ndarray_file(gy_gx_gz[2], gz_fp)

#     del gy_gx_gz

    sys.stderr.write('save %s: %f seconds\n' % (structure, time.time() - t))

except Exception as e:
    sys.stderr.write('%s\n' % e)
    sys.stderr.write('Error computing gradient for %s.\n' % structure)

Gradient 7N: 2.818808 seconds
save 7N: 0.000001 seconds


In [ ]:
display_volume_sections(gy_gx_gz[0])